# Dusty wave

## Imports

In [ ]:
%load_ext blackcellmagic

In [ ]:
import itertools
import pathlib

import numpy as np
import pandas as pd
import plonk

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot, row
from bokeh.palettes import inferno, viridis
from bokeh.plotting import figure
output_notebook()

## Show an example simulation

In [ ]:
# Run
run = 'two species'

# Select species
dust_id = 1

# Select snaps to plot
snap_slice = slice(None, None, 5)

# Select particles to plot
particle_slice = slice(None, None, 1)

# Load data
directory = pathlib.Path(f'~/runs/multigrain/dustywave.fargo3d/{run}').expanduser()
sim = plonk.load_sim(prefix='dustywave', directory=directory)
initial_snap = sim.snaps[0]
snaps = sim.snaps[snap_slice]

# Plot range
x_range = (-0.5, 0.5)
vx_max = np.abs(initial_snap['vx']).max() * 1.05
vx_range = (-vx_max, vx_max)
subsnap = initial_snap[initial_snap['dust_id'] == dust_id]
rho_mean = subsnap['rho'].mean()
rho_var = 1.5e-4
rho_range = (rho_mean * (1 - rho_var), rho_mean * (1 + rho_var))

# Generate figures
fig1 = figure(x_range=x_range, y_range=vx_range, plot_width=400, plot_height=400)
fig2 = figure(x_range=x_range, y_range=rho_range, plot_width=400, plot_height=400)

for snap, color in zip(snaps, inferno(len(snaps))):
    subsnap = snap[snap['dust_id'] == dust_id]
    fig1.scatter(
        subsnap["x"][particle_slice], subsnap["vx"][particle_slice], line_color=color, fill_color=color, size=5
    )
    fig2.scatter(
        subsnap["x"][particle_slice], subsnap["rho"][particle_slice], line_color=color, fill_color=color, size=5
    )
    
show(row(fig1, fig2))

## Do the analysis

Path to data....

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustywave.fargo3d').expanduser()
paths = sorted(list(root_directory.glob('*')))

Calculate $\rho$ and $v_x$ at $x=0$ for each snapshot in each simulation. Each simulation has a data frame.

In [ ]:
A = 1e-4
c_s = 1

x_value = -0.5
dx = 0.01

def do_analysis(sim):
    
    n_snaps = len(sim.snaps)
    n_species = len(sim.snaps[0].properties["grain size"]) + 1

    time = np.array([snap.properties["time"] for snap in sim.snaps])
    rho = np.zeros((n_snaps, n_species))
    vx = np.zeros((n_snaps, n_species))

    rho_0 = np.zeros(n_species)
    snap = sim.snaps[0]
    for dust_id in range(n_species):
        subsnap = snap[snap['dust_id'] == dust_id]
        rho_0[dust_id] = subsnap['rho'].mean()
        
    for idx, snap in enumerate(sim.snaps):
        mask = (snap['x'] < x_value + dx) & (snap['x'] > x_value - dx)
        for dust_id in range(n_species):
            subsnap = snap[(snap["dust_id"] == dust_id) & mask]
            rho[idx, dust_id] = (subsnap["rho"].mean() - rho_0[dust_id]) / (A * rho_0[dust_id])
            vx[idx, dust_id] = subsnap["vx"].mean() / (A * c_s)

    arrays = np.hstack((time[:, np.newaxis], rho, vx))
    columns = (
        ["time"]
        + [f"rho.{idx}" for idx in range(n_species)]
        + [f"vx.{idx}" for idx in range(n_species)]
    )
    
    return pd.DataFrame(arrays, columns=columns)

Loop over each simulation.

In [ ]:
dataframes = dict()

for p in paths:
    sim = plonk.load_sim(prefix="dustywave", directory=p)
    name = "-".join(sim.path.name.split("=")[-1].split("-")[::-1])
    print(f"Analysis for {name}")

    dataframes[name] = do_analysis(sim)
    del sim

## Plot results

Plot the gas density and velocity at $x=0$ for each run.

In [ ]:
x_range = 0.0, 2.0
y_range = -1, 1

p1 = figure(
    title="Normalized gas density at x=0",
    x_range=x_range,
    y_range=y_range,
    plot_width=400,
    plot_height=300,
)
p2 = figure(
    title="Normalized gas velocity at x=0",
    x_range=x_range,
    y_range=y_range,
    plot_width=400,
    plot_height=300,
)

colors = viridis(len(dataframes))

for (name, df), color in zip(dataframes.items(), colors):

    x = np.array(df["time"])
    y = np.array(df["rho.0"])
    p1.line(x, y, line_color=color)
    p1.circle(x, y, legend_label=name, line_color=color, fill_color=None)

    y = np.array(df["vx.0"])
    p2.line(x, y, line_color=color)
    p2.circle(x, y, legend_label=name, line_color=color, fill_color=None)

show(row(p1, p2))

Plot each species density and velocity.

In [ ]:
t_range = 0.0, 2.0
rho_range = -1, 1
vx_range = -1, 1

ps1 = list()
ps2 = list()

for run, df in dataframes.items():

    n_species = len([col for col in df.columns if col.startswith('rho')])
    colors = viridis(n_species)
    
    p1 = figure(
        title=f"Normalized density at x=0 for {run}",
        x_range=t_range,
        y_range=rho_range,
        plot_width=400,
        plot_height=300,
    )
    p2 = figure(
        title=f"Normalized velocity at x=0 for {run}",
        x_range=t_range,
        y_range=vx_range,
        plot_width=400,
        plot_height=300,
    )

    x = np.array(df["time"])

    for idx, color in enumerate(colors):
        legend_label = f"dust {idx}" if idx != 0 else "gas"

        y = np.array(df[f"rho.{idx}"])
        p1.line(x, y, line_color=color)
        p1.circle(x, y, legend_label=legend_label, line_color=color, fill_color=None)

        y = np.array(df[f"vx.{idx}"])
        p2.line(x, y, line_color=color)
        p2.circle(x, y, legend_label=legend_label, line_color=color, fill_color=None)

    ps1.append(p1)
    ps2.append(p2)

ps = list(itertools.chain(*zip(ps1, ps2)))
grid = gridplot(ps, ncols=2)
show(grid)